In [ ]:
import pickle
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import tqdm as notebook_tqdm
import os
from dotenv import load_dotenv
load_dotenv()


In [5]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"})

vectorstore = FAISS.load_local('Schritt3.-FAISS-Vectorstore', embedding_model,allow_dangerous_deserialization=True)

In [8]:
results = vectorstore.similarity_search(
            'hello',
            k=50,)

/Users/lautenschlaeger/Downloads/ZIP-APP/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [1]:
from langchain.agents import tool, create_react_agent, AgentExecutor
#from langgraph.prebuilt import create_react_agent
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import tqdm as notebook_tqdm
import constants

In [2]:
from typing import Union


@tool
def vector_search(query: str) -> str:
    """Durchsuche die Vekordatenbank nach Vektoren, die der Nutzeranfrage am ähnlichsten sind."""
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
    vectorstore = FAISS.load_local("Schritt3.-FAISS-Vectorstore", embedding_model, allow_dangerous_deserialization=True)
    results = vectorstore.similarity_search(query, k=5)
    result_string = ""
    for res in results:
        result_string += f"* {res.page_content} [{res.metadata}]"
    return result_string

@tool
def vergleiche_deckensysteme(input: Union[str, dict]) -> str:
    """Du bekommst die Spannweite und die Nutzlast eines Einfeldträger-Deckensystems übergeben und sollst einen Vergleich der Umweltwirkung, spezifischer dem GWP dieser schaffen. Den Vergleich stellst für ienen besseren Überblick grafisch dar."""
    if isinstance(input, str):
        import json
        try:
            input = json.loads(input)
        except Exception as e:
            return f"❌ Fehler beim Parsen des Inputs: {e}"

    spannweite = input.get("spannweite")
    nutzlast = input.get("nutzlast")

    try:
        spannweite = float(str(spannweite).replace(',', '.'))
        nutzlast = float(str(nutzlast).replace(',', '.'))
    except Exception:
        return "❌ Ungültiger Wert für Spannweite oder Nutzlast."

    dateipfad = "Schritt6.-DataFrame-ExcelTabellen/Predictions_Lineare_Regression_Literatur_generiert.xlsx"
    #prediction_daten = pd.read_excel(dateipfad)
    #prediction_daten.columns = prediction_daten.columns.str.strip()

    bedingungen = [
    #    prediction_daten['Nutzlast'] == nutzlast,
    #    prediction_daten['Spannweite'] == spannweite
    ]
    gesamt_bedingung = bedingungen[0]
    for bed in bedingungen[1:]:
        gesamt_bedingung &= bed

    #ergebnis = prediction_daten[gesamt_bedingung]

    #if ergebnis.empty:
        return f"Keine passenden Deckensysteme für Spannweite {spannweite} m und Nutzlast {nutzlast} kN/m² gefunden."

    #ergebnis_sortiert = ergebnis.sort_values(by="GWP_predicted", ascending=False)
    #normierte_werte = ergebnis_sortiert["GWP_predicted"].values / ergebnis_sortiert["GWP_predicted"].max()

    #relevante_spalten = ['Deckensystem', 'GWP_predicted', 'Spannweite', 'Nutzlast']
    return "test"

In [ ]:
def initialize_agent():
    """Initializes the LangChain agent."""
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        api_key=os.getenv("OPENAI_API_KEY")
    )
    
    prompt = PromptTemplate(
        input_variables=["input", "tools", "tool_names", "agent_scratchpad"],
        template=constants.CHATBOT_PROMPT_TEMPLATE
    )

    tools = [vector_search, vergleiche_deckensysteme]
    
    agent = create_react_agent(
        llm=llm,
        tools=tools,
        prompt=prompt
    )
    
    return AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True)

agent_executor = initialize_agent()


In [4]:
antwort = agent_executor.invoke({"input": 'suche nach beton'})
bot_response = antwort["output"]



> Entering new AgentExecutor chain...
Thought: Ich soll eine Analyse und einen Vergleich von Deckensystemen hinsichtlich ihres Global Warming Potentials (GWP) durchführen. Da der Kontext "beton" ist, könnte eine Vektorsuche nach spezifischen Informationen zu Beton-Deckensystemen hilfreich sein.

Action: vector_search
Action Input: 
```json
{
  "query": "Wie groß ist das GWP einer Stahlbetonhohldiele?"
}
```

/Users/lautenschlaeger/Downloads/ZIP-APP/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/lautenschlaeger/Downloads/ZIP-APP/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


* Stahlbetonunterzüge zum Einsatz. [{'source': '(15)Bautechnik - 2024 - Pfanner - kobilanzierung in der Tragwerks‐ und Fassadenplanung.md'}]* 969-977. # DEUTSCHER AUSSCHUSS FÜR STAHLBETON 

## DAfStb-Richtlinie

## Treibhausgasreduzierte Tragwerke aus Beton, Stahlbeton oder Spannbeton

Teil 1: Deckenbauteile

Entwurf August 2023

Notifiziert gemäß der Richtlinie (EU) 2015/1535 des Europäischen Parlaments und des Rates vom 9. September 2015 über ein Informationsverfahren auf dem Gebiet der technischen Vorschriften und der Vorschriften für die Dienste der Informationsgesellschaft (ABI. [{'source': '(14)Entwicklung eines Benchmarksystems für die grauen emissionen von Deceknsystemen.md'}]* Berlin 19.04.2022. [23] Stefanie Malohing: Konventionelle Stahlbetonbauweise vs. Deckentragwerke aus Spannbetonelementen - Untersuchungen an einem mehrgeschossigen Gebäudekomplex. Masterarbeit. Berlin 31.10.2022. [24] Gabor Attila Callmuth-Jonas: Umbemessung des Aussteifungssystems eines Stahlbetongebäud

/Users/lautenschlaeger/Downloads/ZIP-APP/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


* Stahlbetonunterzüge zum Einsatz. [{'source': '(15)Bautechnik - 2024 - Pfanner - kobilanzierung in der Tragwerks‐ und Fassadenplanung.md'}]* 969-977. # DEUTSCHER AUSSCHUSS FÜR STAHLBETON 

## DAfStb-Richtlinie

## Treibhausgasreduzierte Tragwerke aus Beton, Stahlbeton oder Spannbeton

Teil 1: Deckenbauteile

Entwurf August 2023

Notifiziert gemäß der Richtlinie (EU) 2015/1535 des Europäischen Parlaments und des Rates vom 9. September 2015 über ein Informationsverfahren auf dem Gebiet der technischen Vorschriften und der Vorschriften für die Dienste der Informationsgesellschaft (ABI. [{'source': '(14)Entwicklung eines Benchmarksystems für die grauen emissionen von Deceknsystemen.md'}]* Berlin 19.04.2022. [23] Stefanie Malohing: Konventionelle Stahlbetonbauweise vs. Deckentragwerke aus Spannbetonelementen - Untersuchungen an einem mehrgeschossigen Gebäudekomplex. Masterarbeit. Berlin 31.10.2022. [24] Gabor Attila Callmuth-Jonas: Umbemessung des Aussteifungssystems eines Stahlbetongebäud

...

/Users/lautenschlaeger/Downloads/ZIP-APP/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


* Stahlbetonunterzüge zum Einsatz. [{'source': '(15)Bautechnik - 2024 - Pfanner - kobilanzierung in der Tragwerks‐ und Fassadenplanung.md'}]* 969-977. # DEUTSCHER AUSSCHUSS FÜR STAHLBETON 

## DAfStb-Richtlinie

## Treibhausgasreduzierte Tragwerke aus Beton, Stahlbeton oder Spannbeton

Teil 1: Deckenbauteile

Entwurf August 2023

Notifiziert gemäß der Richtlinie (EU) 2015/1535 des Europäischen Parlaments und des Rates vom 9. September 2015 über ein Informationsverfahren auf dem Gebiet der technischen Vorschriften und der Vorschriften für die Dienste der Informationsgesellschaft (ABI. [{'source': '(14)Entwicklung eines Benchmarksystems für die grauen emissionen von Deceknsystemen.md'}]* Berlin 19.04.2022. [23] Stefanie Malohing: Konventionelle Stahlbetonbauweise vs. Deckentragwerke aus Spannbetonelementen - Untersuchungen an einem mehrgeschossigen Gebäudekomplex. Masterarbeit. Berlin 31.10.2022. [24] Gabor Attila Callmuth-Jonas: Umbemessung des Aussteifungssystems eines Stahlbetongebäud

/Users/lautenschlaeger/Downloads/ZIP-APP/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


* Stahlbetonunterzüge zum Einsatz. [{'source': '(15)Bautechnik - 2024 - Pfanner - kobilanzierung in der Tragwerks‐ und Fassadenplanung.md'}]* 969-977. # DEUTSCHER AUSSCHUSS FÜR STAHLBETON 

## DAfStb-Richtlinie

## Treibhausgasreduzierte Tragwerke aus Beton, Stahlbeton oder Spannbeton

Teil 1: Deckenbauteile

Entwurf August 2023

Notifiziert gemäß der Richtlinie (EU) 2015/1535 des Europäischen Parlaments und des Rates vom 9. September 2015 über ein Informationsverfahren auf dem Gebiet der technischen Vorschriften und der Vorschriften für die Dienste der Informationsgesellschaft (ABI. [{'source': '(14)Entwicklung eines Benchmarksystems für die grauen emissionen von Deceknsystemen.md'}]* Berlin 19.04.2022. [23] Stefanie Malohing: Konventionelle Stahlbetonbauweise vs. Deckentragwerke aus Spannbetonelementen - Untersuchungen an einem mehrgeschossigen Gebäudekomplex. Masterarbeit. Berlin 31.10.2022. [24] Gabor Attila Callmuth-Jonas: Umbemessung des Aussteifungssystems eines Stahlbetongebäud

/Users/lautenschlaeger/Downloads/ZIP-APP/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
#Test Vektordatenbank
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# HuggingFace Embeddings für LangChain verwenden
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)

vectorstore = FAISS.load_local(
    "Schritt3.-FAISS-Vectorstore",
    embeddings=embeddings,
    allow_dangerous_deserialization=True
)

results = vectorstore.similarity_search("Testabfrage", k=1)
print(results)



C:\Users\dusti\AppData\Local\Temp\ipykernel_11916\1522785788.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


[Document(id='22a438de-d38d-4b48-bc41-bd00c574b54c', metadata={'source': '(20)Erfolgspotentiale Holzhybridbauweise.md'}, page_content='BMEL/BMUB.')]
